In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import keras_tuner as kt
import os
import keras

In [8]:
from tensorflow.python.client import device_lib

def gpuon():
    local_device_protos = device_lib.list_local_devices()
    print([x.name for x in local_device_protos if x.device_type == 'GPU'])

gpuon()

['/device:GPU:0']


I0000 00:00:1742363410.908303    4860 gpu_device.cc:2019] Created device /device:GPU:0 with 7537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:06:00.0, compute capability: 8.6


In [9]:
#Definición de directorios:
dataset_dir = os.path.join(os.getcwd(), "dataset")
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")
valid_dir = os.path.join(dataset_dir, "valid")

# Definimos y creamos la carpeta donde se guardarán los modelos.
carpeta_modelos = os.path.join(os.getcwd(), "Modelos")

keras_tuner_dir = os.path.join(carpeta_modelos, "keras_tuner")

os.makedirs(keras_tuner_dir, exist_ok=True)

print(f"Carpeta de modelos creada: {keras_tuner_dir}")

# Directorios de Keras Tuner y TensorBoard
KERAS_TRIALS_DIR = "kerastuner"
KERAS_PROJECT_NAME = "image_classification"
KERAS_PROJECT_TENSORBOARD = "tensorboard_logs"

Carpeta de modelos creada: /home/alvaro/ml/PEC1AP/Modelos/keras_tuner


In [10]:
# Parámetros
batch_size = 32
img_height = 224
img_width = 224
AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(42)  # Establecemos semilla

In [11]:
# Carga de datos de entrenamiento
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode="categorical",
    seed=42,
    shuffle=True)

# Carga de datos de test
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode="categorical",
    seed=42,
    shuffle=False)

val_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode="categorical",
    seed=42,
    shuffle=False
)

Found 7624 files belonging to 53 classes.
Found 265 files belonging to 53 classes.
Found 265 files belonging to 53 classes.


In [12]:
# Normalización
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)

# Número de clases
num_classes = 53

In [6]:
# Función para construir el modelo con Keras Tuner
# Demostramos la superioridad de los modelos convolucionales frente a las redes de perceptrón multicapa.
def build_model(hp):
    model_type = hp.Choice("model_type", ["mlp", "cnn"])  # Permite seleccionar MLP o CNN
    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = inputs

    if model_type == "cnn":
        # Construcción de una CNN con hiperparámetros optimizables
        for layer in range(hp.Int("cnn_layers", 2, 4)):
            x = layers.Conv2D(
                hp.Int(f"filters_{layer}", 16, 128, step=16),
                kernel_size=(3, 3),
                activation=hp.Choice("activation", values=["relu", "tanh"]),
                padding="same")(x)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)

        x = layers.Flatten()(x)

    elif model_type == "mlp":
        # Construcción de una MLP con hiperparámetros optimizables
        x = layers.Flatten()(x)
        for layer in range(hp.Int("mlp_layers", 1, 3)):
            x = layers.Dense(
                hp.Int(f"units_{layer}", 64, 128, step=32),
                activation=hp.Choice("activation", values=["relu", "tanh"]))(x)

    # Dropout antes de la capa densa
    if hp.Boolean("dropout_before_dense"):
        x = layers.Dropout(0.5)(x)

    x = layers.Dense(
        units=hp.Int("dense_units", 64, 128, step=32),
        activation=hp.Choice("activation", values=["relu", "tanh"]))(x)

    # Dropout después de la capa densa
    if hp.Boolean("dropout_after_dense"):
        x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


In [7]:
# Configuración de Keras Tuner con Random Search
tuner = kt.RandomSearch(
    build_model,
    max_trials=10,  # Prueba 10 configuraciones diferentes
    objective="val_accuracy",
    directory=KERAS_TRIALS_DIR,
    project_name=KERAS_PROJECT_NAME,
    overwrite=True
)

In [8]:
# Realiza la búsqueda sin entrenar el modelo
tuner.search(
    train_ds,
    validation_data=test_ds,  # Usa test_ds como validación, pero sin usarlo luego en test
    epochs=10,  # Solo para buscar hiperparámetros
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=KERAS_PROJECT_TENSORBOARD)]
)

Trial 10 Complete [03h 30m 33s]
val_accuracy: 0.47547170519828796

Best val_accuracy So Far: 0.47547170519828796
Total elapsed time: 09h 01m 15s


In [10]:
tuner.results_summary()

Results summary
Results in kerastuner\image_classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 09 summary
Hyperparameters:
model_type: cnn
mlp_layers: 2
units_0: 64
activation: relu
dropout_before_dense: True
dense_units: 128
dropout_after_dense: True
cnn_layers: 3
filters_0: 128
filters_1: 48
units_1: 96
filters_2: 16
Score: 0.47547170519828796

Trial 06 summary
Hyperparameters:
model_type: cnn
mlp_layers: 3
units_0: 64
activation: tanh
dropout_before_dense: True
dense_units: 96
dropout_after_dense: False
cnn_layers: 2
filters_0: 112
filters_1: 16
Score: 0.35849055647850037

Trial 02 summary
Hyperparameters:
model_type: cnn
mlp_layers: 2
units_0: 128
activation: relu
dropout_before_dense: False
dense_units: 96
dropout_after_dense: True
cnn_layers: 2
filters_0: 16
filters_1: 16
Score: 0.03773584961891174

Trial 05 summary
Hyperparameters:
model_type: cnn
mlp_layers: 1
units_0: 64
activation: relu
dropout_before_dense: False
dense_units: 128
dro

In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'model_type': 'cnn', 'mlp_layers': 2, 'units_0': 64, 'activation': 'relu', 'dropout_before_dense': True, 'dense_units': 128, 'dropout_after_dense': True, 'cnn_layers': 3, 'filters_0': 128, 'filters_1': 48, 'units_1': 96, 'filters_2': 16}


In [11]:
#Importamos los resultados para visualizar los trials en un data frame y salvar los resultados en un csv.

trials = tuner.oracle.get_best_trials(num_trials=10)  # Cambia el número según lo que quieras ver

data = []
for trial in trials:
    trial_data = trial.hyperparameters.values
    trial_data["score"] = trial.score  # Agrega el score del modelo
    data.append(trial_data)

df = pd.DataFrame(data)

# Exportar el DataFrame a un archivo CSV
df.to_csv("trials_data.csv", index=False)

print(df)

  model_type  mlp_layers  units_0 activation  dropout_before_dense  \
0        cnn           2       64       relu                  True   
1        cnn           3       64       tanh                  True   
2        cnn           2      128       relu                 False   
3        cnn           1       64       relu                 False   
4        mlp           2       64       tanh                 False   
5        mlp           1       96       relu                 False   
6        mlp           1      128       relu                  True   
7        cnn           1       64       relu                  True   
8        mlp           1       96       relu                 False   
9        mlp           1       96       relu                 False   

   dense_units  dropout_after_dense  cnn_layers  filters_0  filters_1  \
0          128                 True         3.0      128.0       48.0   
1           96                False         2.0      112.0       16.0   
2         

In [1]:
#Visualización de estadísticas en Tensorboard
!tensorboard --logdir=tensorboard_logs

^C


De nuevo, vamos a proceder a la búsqueda de la mejor configuración centrándonos en redes CNN limitando a 2-4 capas de convolución y 1-2 capas densas.

In [ ]:
def build_model(hp):
    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = inputs

    # Construcción de una CNN con hiperparámetros optimizables
    for layer in range(hp.Int("cnn_layers", 2, 4)):  # Entre 2 y 4 capas convolucionales
        x = layers.Conv2D(
            filters=hp.Int(f"filters_{layer}", 16, 128, step=16),
            kernel_size=(3, 3),
            activation=hp.Choice("activation", values=["relu", "tanh"]),
            padding="same",
            kernel_regularizer=regularizers.l2(hp.Float("l2_conv", 1e-5, 1e-2, sampling="LOG")) # L2 regularization
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)

        # Dropout en las capas convolucionales con valor configurable
        if hp.Boolean("dropout_conv"):
            x = layers.Dropout(hp.Float("dropout_rate_conv", 0.2, 0.5, step=0.1))(x)

    x = layers.Flatten()(x)

    # Capas densas limitadas entre 1 y 2
    for layer in range(hp.Int("dense_layers", 1, 2)):
        x = layers.Dense(
            units=hp.Int(f"dense_units_{layer}", 64, 256, step=64),
            activation=hp.Choice("activation_dense", values=["relu", "tanh"]),
            kernel_regularizer=regularizers.l2(hp.Float("l2_dense", 1e-5, 1e-2, sampling="LOG")) # L2 regularization
        )(x)
        
        # Dropout después de la capa densa con valor configurable
        if hp.Boolean("dropout_dense"):
            x = layers.Dropout(hp.Float("dropout_rate_dense", 0.2, 0.5, step=0.1))(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs=inputs, outputs=outputs)

    # Optimización con Adam, RMSprop o SGD
    model.compile(
        optimizer=hp.Choice("optimizer", ["adam", "rmsprop", "sgd"]),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

# Configuración de Keras Tuner con Random Search
tuner = kt.RandomSearch(
    build_model,
    max_trials=20,  # Aumentamos el número de configuraciones probadas
    objective="val_accuracy",
    directory=KERAS_TRIALS_DIR,
    project_name=KERAS_PROJECT_NAME,
    overwrite=True
)

# Realiza la búsqueda de hiperparámetros
tuner.search(
    train_ds,
    validation_data=test_ds,
    epochs=15,  # Aumentamos las épocas para capturar más variabilidad
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=KERAS_PROJECT_TENSORBOARD)]
)

Nueva búsqueda de hiperparámetros más centrada en los dos mejores resultados de la busqueda anterior. 

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt

# Función para construir el modelo basado en los dos mejores modelos
def build_model_best_hps(hp):
    # Valores FIJOS
    model_type = "cnn"  # Ambos modelos eran CNN
    units_0 = 64  # Ambos usaban 64 unidades en la primera capa densa
    dropout_before_dense = True  # Ambos aplicaban dropout antes de la capa densa

    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = inputs

    if model_type == "cnn":
        # Número de capas convolucionales (variable: 2 o 3)
        cnn_layers = hp.Choice("cnn_layers", [2, 3])
        
        # Construcción de capas convolucionales
        for layer in range(cnn_layers):
            filters = hp.Choice(f"filters_{layer}", [128, 112] if layer == 0 else [48, 16])
            x = layers.Conv2D(
                filters,
                kernel_size=(3, 3),
                activation=hp.Choice("activation", values=["relu", "tanh"]),  # Variable
                padding="same")(x)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)

        x = layers.Flatten()(x)

    # Dropout antes de la capa densa (fijo en True)
    x = layers.Dropout(0.5)(x)

    # Capa densa con unidades variables (128 o 96)
    x = layers.Dense(
        units=hp.Choice("dense_units", [128, 96]),
        activation=hp.Choice("activation", values=["relu", "tanh"]))(x)

    # Dropout después de la capa densa (variable: True o False)
    if hp.Boolean("dropout_after_dense"):
        x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [7]:
# Configuración del tuner nuevamente con los directorios definidos

tuner = kt.Hyperband(
    build_model_best_hps,
    objective="val_accuracy",
    max_epochs=15,  # Más épocas para mejorar el modelo
    directory=KERAS_TRIALS_DIR,
    project_name=KERAS_PROJECT_NAME
)

In [8]:
# Realiza la búsqueda sin entrenar el modelo
tuner.search(
    train_ds,
    validation_data=test_ds,
    epochs=10,  # Solo para buscar hiperparámetros
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=KERAS_PROJECT_TENSORBOARD)]
)

Trial 30 Complete [00h 43m 00s]
val_accuracy: 0.5622641444206238

Best val_accuracy So Far: 0.8377358317375183
Total elapsed time: 08h 43m 26s


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'cnn_layers': 3, 'filters_0': 112, 'activation': 'relu', 'filters_1': 48, 'dense_units': 128, 'dropout_after_dense': False, 'filters_2': 16, 'tuner/epochs': 15, 'tuner/initial_epoch': 5, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}


In [10]:
trials = tuner.oracle.get_best_trials(num_trials=10)

data = []
for trial in trials:
    trial_data = trial.hyperparameters.values
    trial_data["score"] = trial.score
    data.append(trial_data)

df = pd.DataFrame(data)

df.to_csv("trials_data.csv", index=False)

print(df)

   cnn_layers  filters_0 activation  filters_1  dense_units  \
0           3        112       relu         48          128   
1           2        128       relu         16          128   
2           2        128       relu         16          128   
3           3        112       relu         16           96   
4           2        128       relu         16           96   
5           3        112       relu         48          128   
6           2        128       relu         16          128   
7           2        128       relu         16          128   
8           3        128       relu         48          128   
9           2        112       relu         16           96   

   dropout_after_dense  filters_2  tuner/epochs  tuner/initial_epoch  \
0                False         16            15                    5   
1                False         48            15                    5   
2                False         16            15                    5   
3                F

In [11]:
# Callback para visualizar en TensorBoard
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=KERAS_PROJECT_TENSORBOARD)

In [16]:
# Construcción del modelo basado en los mejores hiperparámetros.
# Puede entrenarse el modelo seleccionado pero sin cerrar la sesión. Como la cerramos debemos crear el modelo a partir de los mejores hiperparámetros.

import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential

model = Sequential([
    Input(shape=(224, 224, 3)),  # Definir la entrada correctamente
    layers.Conv2D(112, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(48, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(53, activation='softmax')
])

# Compilamos el modelo
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

# Mostramos el resumen del modelo
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 112)  │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 224, 224, 112)  │           448 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 112)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 48)   │        48,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 112, 112, 48)   │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 16)     │         6,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 53)             │         6,837 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,671,797 (6.38 MB)

 Trainable params: 1,671,445 (6.38 MB)

 Non-trainable params: 352 (1.38 KB)

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=15,  
                               min_delta=0.01,  
                               restore_best_weights=True)

checkpoint = ModelCheckpoint('Modelos/keras_tuner_dir/keras_tuner_model.keras',
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True)

# Entrenamiento
keras_tuner_model_cartas = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[checkpoint, early_stopping])

Epoch 1/50


I0000 00:00:1742363996.726892    5371 service.cc:152] XLA service 0x32a04930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742363996.726960    5371 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-03-19 06:59:56.835184: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742363997.157273    5371 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/239 ━━━━━━━━━━━━━━━━━━━━ 1:11:03 18s/step - accuracy: 0.0000e+00 - auc: 0.4957 - loss: 6.3569 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1742364005.807575    5371 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


239/239 ━━━━━━━━━━━━━━━━━━━━ 45s 113ms/step - accuracy: 0.1364 - auc: 0.7188 - loss: 3.8294 - precision: 0.3672 - recall: 0.0434 - val_accuracy: 0.1547 - val_auc: 0.7090 - val_loss: 4.9527 - val_precision: 0.2692 - val_recall: 0.1057
Epoch 2/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - accuracy: 0.4157 - auc: 0.9240 - loss: 2.1007 - precision: 0.7205 - recall: 0.2823 - val_accuracy: 0.5019 - val_auc: 0.9534 - val_loss: 1.6612 - val_precision: 0.7769 - val_recall: 0.3547
Epoch 3/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - accuracy: 0.5541 - auc: 0.9537 - loss: 1.5998 - precision: 0.8289 - recall: 0.4396 - val_accuracy: 0.6075 - val_auc: 0.9606 - val_loss: 1.4612 - val_precision: 0.8707 - val_recall: 0.4830
Epoch 4/50
239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - accuracy: 0.6610 - auc: 0.9717 - loss: 1.2023 - precision: 0.8729 - recall: 0.5665 - val_accuracy: 0.6981 - val_auc: 0.9702 - val_loss: 1.1945 - val_precision: 0.8342 - val_recall: 0.6264
Epoch 5/50
239/239 ━━━━━━━━━━━━━━━

Nos arroja una buena relación precisión-pérdida.
Hemos guardado el modelo y lo probaremos frente a los datos de test y validación en el fichero Identification_cards_type